## Billboard Lyrics: 1964 - 2015  

*The first 2 cells below are for Google Drive authentication (Colab Notebooks)*  
  *Feel free to ignore*  

### Google Drive authentication  

In [0]:
# source: https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
# need to run every time (even after timeouts)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# ---- imports ---- #
import numpy as np
import pandas as pd

In [0]:
# additional imports 
import math
import re

In [0]:
# directory path for file(s); change as necessary
path = "/content/drive/My Drive/data/"

*You do not really need to create the full-file dataframe for the dataframe compare.*  

In [0]:
# file details (kaggle data set)
file = "billboard_lyrics_1964-2015.csv"
path_file = path + file
df_bb = pd.read_csv(path_file, sep=",", encoding='latin1')

In [0]:
df_bb.shape

(5100, 6)

In [0]:
# file details - very small subset of original file
file = "billboard_diff1.csv"
path_file = path + file
df_diff1 = pd.read_csv(path_file, sep=",", encoding='latin1')

In [0]:
df_diff1.shape

(12, 6)

In [0]:
df_diff1

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1
2,3,i cant get no satisfaction,the rolling stones,1965,,1
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1
5,6,downtown,petula clark,1965,when youre alone and life is making you lonel...,1
6,7,help,the beatles,1965,help i need somebody help not just anybody hel...,3
7,11,help me rhonda,the beach boys,1965,well since she put me down i ve been out doin ...,3
8,12,king of the road,roger miller,1965,trailer for sale or rent rooms to let fifty c...,1
9,13,the birds and the bees,jewel akens,1965,let me tell ya bout the birds and the bees and...,3


In [0]:
# file details - very small subset of original file; mocked up to create differences
file = "billboard_diff2.csv"
path_file = path + file
df_diff2 = pd.read_csv(path_file, sep=",", encoding='latin1')

In [0]:
df_diff2.shape

(13, 6)

In [0]:
df_diff2

,Rank,Song,Artist,Year,Lyrics,Source
0,3,i cant get no satisfaction,the rolling stones,1965,,1
1,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1
2,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1
3,6,downtown,petula clark,1965,when youre alone and life is making you lonel...,1
4,7,help,the beatles,1965,help i need somebody help not just anybody hel...,3
5,8,cant you hear my heart beat,hermans hermits,1965,carterlewis every time i see you lookin my way...,5
6,9,crying in the chapel,elvis presley,1965,you saw me crying in the chapel the tears i s...,1
7,10,my girl,the temptations,1965,ive got sunshine on a cloudy day when its cold...,3
8,11,help me rhonda,the beach boys,1965,well since she put me down i ve been out doin ...,3
9,12,king of the road,robert miller,1965,trailer for sale or rent rooms to let fifty c...,1


**Define Functions**  

In [0]:
# compare dataframes on a row level, based on business keys
def dataframe_compare_row_differences(df1r, df2r, cols_r):
  '''dataframe compare on row level; business keys join'''

  # get rows that only exist in each dataframe, based on a business key(s) join
  df1_row_only = pd.merge(df1r, df2r.loc[:, cols_r], left_on=cols_r, right_on=cols_r, how='left', indicator=True).query('_merge == "left_only"')
  df2_row_only = pd.merge(df1r.loc[:, cols_r], df2r, left_on=cols_r, right_on=cols_r, how='right', indicator=True).query('_merge == "right_only"')
  # rename _merge column to _flag
  df1_row_only.rename(columns={'_merge': '_flag'}, inplace=True)
  df2_row_only.rename(columns={'_merge': '_flag'}, inplace=True)
  # populate _flag column if dataframe is not empty
  if not df1_row_only.empty: df1_row_only['_flag'] = "row_left"
  if not df2_row_only.empty: df2_row_only['_flag'] = "row_right"
  # combine (concat) dataframes for 1 dataframe with all full row differences
  df_diffs_row = pd.concat([df1_row_only, df2_row_only], sort=False)
  # create dataframes for original dataframes where the business key(s) match -- for elementwise comparison later (if needed)
  df1_keys_match = pd.merge(df1r, df1_row_only.loc[:, cols_r], left_on=cols_r, right_on=cols_r, how='left', indicator=True).query('_merge == "left_only"')
  df2_keys_match = pd.merge(df2r, df2_row_only.loc[:, cols_r], left_on=cols_r, right_on=cols_r, how='left', indicator=True).query('_merge == "left_only"')

  return df_diffs_row, df1_keys_match, df2_keys_match

In [0]:
# compare dataframes elementwise
def dataframe_compare_element_differences(df1e, df2e, cols_e):
  '''dataframe compare elementwise'''

  # prep dataframes for elementwise comparison; sort by key(s), drop index, drop _merge column
  df1e = df1e.sort_values(by=cols_e).reset_index(drop=True)
  df2e = df2e.sort_values(by=cols_e).reset_index(drop=True)
  if '_merge' in df1e:
    df1e = df1e.drop(columns=['_merge'])
  if '_merge' in df2e:
    df2e = df2e.drop(columns=['_merge'])
  # create dataframes that show what elements are different between them
  df1e_diff_elm = df1e[df1e.ne(df2e)].dropna(how='all')
  df2e_diff_elm = df2e[df1e.ne(df2e)].dropna(how='all')
  # create _flag column; initialize with np.nan
  df1e_diff_elm['_flag'] = np.nan
  df1e_diff_elm['_flag'] = np.nan 
  # populate _flag column if dataframe is not emtpy (if 1 is not empty, 2 won't be either)
  if not df1e_diff_elm.empty: 
    df1e_diff_elm['_flag'] = "elm_left"
    df2e_diff_elm['_flag'] = "elm_right"
  # combine (concat) dataframes for 1 dataframe with all element differences
  df_diffs_elm = pd.concat([df1e_diff_elm, df2e_diff_elm], sort=False)

  return df_diffs_elm

In [0]:
# compare dataframes for all differences (rows and elements)
def dataframe_compare_all_differences(df1a, df2a, cols_a):
  '''dataframe compare all differences; call separate functions for rows and elements'''

  # call row_differences function
  df_diff_rows, df1_match_keys, df2_match_keys = dataframe_compare_row_differences(df1a, df2a, cols_a)
  # call elementwise_differences function
  df_diff_elms = dataframe_compare_element_differences(df1_match_keys, df2_match_keys, cols_a)
  # combine (concat) dataframes to have all differences together
  df_diff_rows_elms = pd.concat([df_diff_rows, df_diff_elms])
  # sort dataframe
  df_diff_rows_elms.sort_values(by=[*cols_a, '_flag'])
  # dataframe with all differences and values (full outer join)
  df_diff_all_vals = pd.merge(df1a, df2a, how='outer', indicator=True).query('_merge != "both"')
  
  return df_diff_rows_elms, df_diff_all_vals, df1_match_keys, df2_match_keys


In [0]:
# check for all dataframe compare differences
key_cols = ['Year', 'Rank'] # business key(s)
if df_diff1.equals(df_diff2):
  print("Data sets match completely.")
else:
  # call 'all differences' function
  df_diffs_all, df_diffs_all_vals, df_match_keys_left, df_match_keys_right = dataframe_compare_all_differences(df_diff1, df_diff2, key_cols)
  # show 'all differences' dataframe head
  df_diffs_rows = len(df_diffs_all)
  print(f"Row count of differences (full rows and column level): {df_diffs_rows}\n")
  print("Sample (head) of differences:")
  print(df_diffs_all.head())

Row count of differences (full rows and column level): 13

Sample (head) of differences:
    Rank                                      Song  ... Source      _flag
0    1.0                               wooly bully  ...    3.0   row_left
1    2.0  i cant help myself sugar pie honey bunch  ...    1.0   row_left
9   13.0                    the birds and the bees  ...    3.0   row_left
9    8.0               cant you hear my heart beat  ...    5.0  row_right
10   9.0                      crying in the chapel  ...    1.0  row_right

[5 rows x 7 columns]


In [0]:
# placeholder to view all or different portions of df_diffs_all (e.g., .tail(), .sample(n))
df_diffs_all

,Rank,Song,Artist,Year,Lyrics,Source,_flag
0,1.0,wooly bully,sam the sham and the pharaohs,1965.0,sam the sham miscellaneous wooly bully wooly b...,3.0,row_left
1,2.0,i cant help myself sugar pie honey bunch,four tops,1965.0,sugar pie honey bunch you know that i love yo...,1.0,row_left
9,13.0,the birds and the bees,jewel akens,1965.0,let me tell ya bout the birds and the bees and...,3.0,row_left
9,8.0,cant you hear my heart beat,hermans hermits,1965.0,carterlewis every time i see you lookin my way...,5.0,row_right
10,9.0,crying in the chapel,elvis presley,1965.0,you saw me crying in the chapel the tears i s...,1.0,row_right
11,10.0,my girl,the temptations,1965.0,ive got sunshine on a cloudy day when its cold...,3.0,row_right
12,19.0,the birds and the bees,jewel akens,1965.0,let me tell ya bout the birds and the bees and...,3.0,row_right
6,NaN,NaN,roger miller,NaN,NaN,NaN,elm_left
7,NaN,hold me thrill me kiss me,NaN,NaN,NaN,NaN,elm_left
8,NaN,NaN,NaN,NaN,NaN,3.0,elm_left


In [0]:
# placeholder to view all differences with all values populated (not just non-matching)
#  outer join; non-matching rows
df_diffs_all_vals

,Rank,Song,Artist,Year,Lyrics,Source,_merge
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3,left_only
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1,left_only
8,12,king of the road,roger miller,1965,trailer for sale or rent rooms to let fifty c...,1,left_only
9,13,the birds and the bees,jewel akens,1965,let me tell ya bout the birds and the bees and...,3,left_only
10,14,hold me thrill me kiss me,mel carter,1965,hold me hold me never let me go until youve t...,1,left_only
11,15,shotgun,junior walker the all stars,1965,i said Ì¢shotgun shoot em for he runs now do t...,3,left_only
12,8,cant you hear my heart beat,hermans hermits,1965,carterlewis every time i see you lookin my way...,5,right_only
13,9,crying in the chapel,elvis presley,1965,you saw me crying in the chapel the tears i s...,1,right_only
14,10,my girl,the temptations,1965,ive got sunshine on a cloudy day when its cold...,3,right_only
15,12,king of the road,robert miller,1965,trailer for sale or rent rooms to let fifty c...,1,right_only
